## Get Topography data for the region
Currently run in `shakemap` env

In [ ]:
## Get Topography
from bmi_topography import Topography
import os

# rgn in pygmt order
rgn = [-140.8138, -138.3731, 59.4847, 60.9591]

# Output event directory 
dir = '/Users/hyin/shakemap_profiles/default/data/us6000rsy1'
# check if the topo dir already exists, if not create it
if not os.path.exists(dir + '/topo'):
    os.makedirs(dir + '/topo')

## Set bounds of dem you'd like to download
params = Topography.DEFAULT.copy()
params["south"] = rgn[2]
params["north"] = rgn[3]
params["west"] = rgn[0]
params["east"] = rgn[1]
params["dem_type"] ='SRTM15Plus'  # 'SRTM15Plus'   'SRTMGL1'
params["api_key"]= 'df3611483edd8184c6bca8c5c2f5f738'

dem = Topography(**params).load()

## Write dem to `grd` file


In [12]:
import xarray as xr
import rioxarray as rx
import numpy as np
import pygmt

# Write to geotiff
file = 'dem.tif'
dem.rio.to_raster(dir + '/topo/' + file)

# Read it back in and write to .grd file
data = rx.open_rasterio(dir + '/topo/' + file)
z = np.squeeze(data.load().to_numpy())
x = np.squeeze(data['x'].to_numpy())
y = np.squeeze(data['y'].to_numpy())
X,Y = np.meshgrid(x,y)
if np.shape(X)==np.shape(z):
    grid = pygmt.xyz2grd(x=X.flatten(),y=Y.flatten(),z=z.flatten(), outgrid=f"{dir}/topo/dem.grd",
                         spacing=(np.abs(x[1]-x[0]), np.abs(y[1]-y[0])), 
                         region=[np.min(x),np.max(x),np.min(y),np.max(y)])

## Calculate topography gradient

In [3]:
## Plot Topography 
# calculate the reflection of a light source projecting from west to east
# (azimuth of 270 degrees) and at a latitude of 30 degrees from the horizon
dgrid = pygmt.grdgradient(grid='dem.grd', radiance=[270, 30])


In [4]:
z = np.squeeze(dgrid.load().to_numpy())
x = np.squeeze(dgrid['x'].to_numpy())
y = np.squeeze(dgrid['y'].to_numpy())
X,Y = np.meshgrid(x,y)
if np.shape(X)==np.shape(z):
    grid = pygmt.xyz2grd(x=X.flatten(),y=Y.flatten(),z=z.flatten(), outgrid='dem_grad.grd',
                         spacing=(np.abs(x[1]-x[0]), np.abs(y[1]-y[0])), 
                         region=[np.min(x),np.max(x),np.min(y),np.max(y)])